TASK 3

In [24]:
# langgraph_parallel_llm_agent.py
#
# LangGraph fan-out / fan-in with parallel LLM execution.
# - get_user_input -> fanout_models -> call_llama & call_qwen -> print_both_responses -> loop
# - 'verbose' / 'quiet' toggle for tracing
# - empty input loops back to get_user_input (never passed to LLM)
# - safe fallbacks so script runs in Colab without tokens

import sys
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Tuple

# -------------------------
# Device selection
# -------------------------
def get_device():
    if torch.cuda.is_available():
        print("Using CUDA (NVIDIA GPU)")
        return "cuda"
    elif torch.backends.mps.is_available():
        print("Using MPS (Apple Silicon)")
        return "mps"
    else:
        print("Using CPU")
        return "cpu"

# -------------------------
# State
# -------------------------
class AgentState(TypedDict):
    user_input: str
    should_exit: bool
    llama_response: str
    qwen_response: str
    verbose: bool

# -------------------------
# LLM loader with safe langchain fix
# -------------------------
def load_llm_wrapped(model_id: str, device: str):
    """
    Load a HF model and wrap in a simple adapter exposing .invoke(prompt) -> str
    Adds minimal defensive handling for langchain top-level attributes.
    """
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            dtype=torch.float16 if device != "cpu" else torch.float32,
            device_map=device if device == "cuda" else None,
        )
        if device == "mps":
            model = model.to(device)

        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )

        # Defensive langchain attributes to avoid AttributeError in some installs
        try:
            import langchain
            if not hasattr(langchain, "verbose"):
                langchain.verbose = False
            if not hasattr(langchain, "debug"):
                langchain.debug = False
            if not hasattr(langchain, "llm_cache"):
                langchain.llm_cache = None
        except Exception:
            # non-fatal
            pass

        wrapped = HuggingFacePipeline(pipeline=pipe)

        # Produce a .invoke(prompt) -> str behaviour
        class Adapter:
            def __init__(self, llm):
                self.llm = llm
            def invoke(self, prompt: str) -> str:
                # HuggingFacePipeline.invoke returns a string in many setups,
                # but to be safe, handle both dict/list shapes.
                out = self.llm.invoke(prompt)
                # if it's already a string, return it
                if isinstance(out, str):
                    return out
                # if it's a dict or list, try to extract the text
                try:
                    # common shapes: {"generated_text": "..."} or [{"generated_text": "..."}]
                    if isinstance(out, list) and len(out) > 0 and isinstance(out[0], dict):
                        if "generated_text" in out[0]:
                            return out[0]["generated_text"]
                    if isinstance(out, dict) and "generated_text" in out:
                        return out["generated_text"]
                except Exception:
                    pass
                return str(out)

        return Adapter(wrapped)

    except Exception as e:
        # Re-raise to let caller decide fallback; include original for diagnostics
        raise RuntimeError(f"Failed to load {model_id}: {e}") from e

# -------------------------
# Create models (attempt Llama & Qwen; fallback small model used if Qwen fails)
# -------------------------
def create_models() -> Tuple[object, object]:
    device = get_device()

    # Attempt to load Llama (may be large; if unavailable we'll fallback later)
    llama_id = "meta-llama/Llama-3.2-1B-Instruct"
    llama_llm = None
    try:
        print(f"Loading Llama: {llama_id} ...")
        llama_llm = load_llm_wrapped(llama_id, device)
        print("Llama loaded.")
    except Exception as e:
        print(f"Could not load Llama ({llama_id}): {e}")
        llama_llm = None

    # Attempt to load Qwen (preferred); if fails, fall back to small local model (gpt2)
    qwen_id = "Qwen/Qwen2.5-1.5B-Instruct"
    qwen_llm = None
    try:
        print(f"Loading Qwen: {qwen_id} ...")
        qwen_llm = load_llm_wrapped(qwen_id, device)
        print("Qwen loaded.")
    except Exception as e:
        print(f"Could not load Qwen ({qwen_id}): {e}")
        print("Falling back to a lightweight local model (gpt2) for the Qwen branch (no tokens required).")
        # fallback small model
        fallback_id = "gpt2"
        try:
            qwen_llm = load_llm_wrapped(fallback_id, device)
            print("Fallback (gpt2) loaded for Qwen branch.")
        except Exception as e2:
            print(f"Failed to load fallback model {fallback_id}: {e2}")
            # final fallback: an object that returns an informative message
            class QwenFallback:
                def invoke(self, prompt: str) -> str:
                    return "[QWEN-FALLBACK] No local model available. Install transformers and a small model like 'gpt2'."
            qwen_llm = QwenFallback()

    # If Llama failed to load earlier, use the qwen fallback for both branches (so graph still runs)
    if llama_llm is None:
        print("Using the Qwen/fallback model for the Llama branch as well (Llama unavailable).")
        llama_llm = qwen_llm

    return llama_llm, qwen_llm

# -------------------------
# Build the graph
# -------------------------
def create_graph(llama_llm, qwen_llm):
    def get_user_input(state: AgentState) -> dict:
        if state.get("verbose", False):
            print("[TRACE] Entering get_user_input")
        print("\n" + "=" * 50)
        print("Enter text (or 'quit' to exit). Type 'verbose' or 'quiet' to toggle tracing.")
        print("=" * 50)
        print("> ", end="")
        user_input = input()

        lowered = user_input.strip().lower()
        if lowered == "verbose":
            print("Verbose tracing enabled.")
            return {"user_input": user_input, "should_exit": False, "verbose": True}
        if lowered == "quiet":
            print("Verbose tracing disabled.")
            return {"user_input": user_input, "should_exit": False, "verbose": False}

        if lowered in ("quit", "exit", "q"):
            print("Goodbye!")
            return {"user_input": user_input, "should_exit": True}

        # normal input (possibly empty) – router will handle empty vs non-empty
        return {"user_input": user_input, "should_exit": False}

    def route_after_input(state: AgentState) -> str:
        # three-way branch: END, back-to-input (if empty), or fanout
        if state.get("verbose", False):
            print("[TRACE] route_after_input:", {"should_exit": state.get("should_exit"), "user_input": repr(state.get("user_input"))})
        if state.get("should_exit", False):
            return END
        if str(state.get("user_input", "")).strip() == "":
            if state.get("verbose", False):
                print("[TRACE] empty input -> looping back to get_user_input")
            print("[NOTICE] Empty input received — please type something.")
            return "get_user_input"
        return "fanout_models"

    def fanout_models(state: AgentState) -> dict:
        if state.get("verbose", False):
            print("[TRACE] fanout_models (no-op node used for structural fanout)")
        # no state changes here; edges fan to two child nodes
        return {}

    def call_llama(state: AgentState) -> dict:
        if state.get("verbose", False):
            print("[TRACE] call_llama invoked")
        prompt = f"User: {state['user_input']}\nAssistant:"
        response = llama_llm.invoke(prompt)
        return {"llama_response": response}

    def call_qwen(state: AgentState) -> dict:
        if state.get("verbose", False):
            print("[TRACE] call_qwen invoked")
        prompt = f"User: {state['user_input']}\nAssistant:"
        response = qwen_llm.invoke(prompt)
        return {"qwen_response": response}

    def print_both_responses(state: AgentState) -> dict:
        if state.get("verbose", False):
            print("[TRACE] print_both_responses invoked")
        print("\n" + "=" * 60)
        print("🦙 LLaMA Response")
        print("=" * 60)
        print(state.get("llama_response", "[No Llama response]"))
        print("\n" + "=" * 60)
        print("🧠 Qwen Response")
        print("=" * 60)
        print(state.get("qwen_response", "[No Qwen response]"))
        # clear responses so next iteration starts clean
        return {"llama_response": "", "qwen_response": ""}

    # construct graph
    graph = StateGraph(AgentState)
    graph.add_node("get_user_input", get_user_input)
    graph.add_node("fanout_models", fanout_models)
    graph.add_node("call_llama", call_llama)
    graph.add_node("call_qwen", call_qwen)
    graph.add_node("print_both_responses", print_both_responses)

    graph.add_edge(START, "get_user_input")
    graph.add_conditional_edges(
        "get_user_input",
        route_after_input,
        {
            "fanout_models": "fanout_models",
            "get_user_input": "get_user_input",
            END: END,
        },
    )

    # fan-out to both models
    graph.add_edge("fanout_models", "call_llama")
    graph.add_edge("fanout_models", "call_qwen")

    # fan-in: both model nodes -> aggregator
    graph.add_edge("call_llama", "print_both_responses")
    graph.add_edge("call_qwen", "print_both_responses")

    # loop back
    graph.add_edge("print_both_responses", "get_user_input")

    return graph.compile()

# -------------------------
# Main
# -------------------------
def main():
    print("=" * 60)
    print("LangGraph Parallel LLM Agent (LLaMA + Qwen/fallback)")
    print("=" * 60)

    llama_llm, qwen_llm = create_models()

    graph = create_graph(llama_llm, qwen_llm)

    initial_state: AgentState = {
        "user_input": "",
        "should_exit": False,
        "llama_response": "",
        "qwen_response": "",
        "verbose": False,
    }

    graph.invoke(initial_state)

if __name__ == "__main__":
    # create_models is defined above; call main
    main()

LangGraph Parallel LLM Agent (LLaMA + Qwen/fallback)
Using CUDA (NVIDIA GPU)
Loading Llama: meta-llama/Llama-3.2-1B-Instruct ...


Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

Llama loaded.
Loading Qwen: Qwen/Qwen2.5-1.5B-Instruct ...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

 What is soccer?


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🦙 LLaMA Response
User: What is soccer?
Assistant: You're thinking of the beautiful game, my friend! Soccer, also known as football in most of the world, is a global sport that involves two teams of 11 players trying to score goals by kicking or heading the ball into the opponent's goal. The objective is simple: score more goals than the opposing team by using your skills, strategy, and teamwork to outmaneuver them. It's a thrilling and physically demanding sport that's played all over the world, with a rich history and cultural significance. Are you ready to give it a try?

🧠 Qwen Response
User: What is soccer?
Assistant: Soccer, also known as football or soccer, is a popular team sport played between two teams of eleven players each. The objective of the game is to score more goals than your opponent by kicking a ball into the opposing team's goal.

Key features include:

1. **Objective**: Score more goals than your opponents.
2. **Field Size**: A standard field measures about 100-13

 quit


Goodbye!
